In [5]:
# check subject feature values
curr_data[curr_data['Cerebellum_Cortex_r']>0.1]
#max(curr_data['Cerebellum_Cortex_r']) # sub_0073
def peep_sub(data, sub_name, features):
    print(sub_name,': ')
    for x in features:
        print('\t' ,x, ': ', data.loc[sub_name, x])
        
peep_sub(curr_data, 'sub_0027', [ 'eTIV', 'Cerebellum_Cortex', 'Cerebellum_Cortex_r'])
peep_sub(curr_data, 'sub_0012', [ 'eTIV', 'Cerebellum_Cortex', 'Cerebellum_Cortex_r'])

sub_0027 : 
	 eTIV :  1678632.04462
	 Cerebellum_Cortex :  91251.79999999999
	 Cerebellum_Cortex_r :  0.05436081140739637
sub_0012 : 
	 eTIV :  1323749.89026
	 Cerebellum_Cortex :  119750.2
	 Cerebellum_Cortex_r :  0.09046285924637898


In [ ]:
# one way ANOVA
f_res=stats.f_oneway(tmp_data['Cerebellum-Cortex_AgeSexRemoved'][tmp_data['diagnosis'] == 'PD'],
                     tmp_data['Cerebellum-Cortex_AgeSexRemoved'][tmp_data['diagnosis'] == 'NC'])
"""
The function below was created specifically for the one-way ANOVA table results returned for Type II sum of squares
"""
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']

    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])

    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])

    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov
print(rp.summary_cont(tmp_data['Cerebellum-Cortex_r'].groupby(tmp_data['diagnosis'])))
print(rp.summary_cont(tmp_data['Cerebellum-White-Matter_r'].groupby(tmp_data['diagnosis'])))

In [ ]:
# simpler model also works, and log scale works well for volumes.
def comp_models(data, var_):
    import numpy as np
    dat_ = data.copy(); var_log = var_+'_log';
    dat_[var_+'_log'] = np.log10(dat_[var_]); dat_['eTIV_log'] = np.log10(dat_['eTIV']);
    f1 = var_log +'~C(diagnosis, Treatment(reference="NC"))'; m1 = smf.glm(formula=f1, data=dat_); r1 = m1.fit();
    f2 = var_log +'~sex+C(diagnosis, Treatment(reference="NC"))'; m2 = smf.glm(formula=f2, data=dat_); r2 = m2.fit();
    f3 = var_log +'~sex+age+C(diagnosis, Treatment(reference="NC"))'; m3 = smf.glm(formula=f3, data=dat_); r3 = m3.fit();
    f4 = var_log +'~sex+age+eTIV_log+C(diagnosis, Treatment(reference="NC"))'; m4 = smf.glm(formula=f4, data=dat_); r4 = m4.fit();
    f5 = var_log +'~age+sex*eTIV_log+C(diagnosis, Treatment(reference="NC"))'; m5 = smf.glm(formula=f5, data=dat_); r5 = m5.fit();
    f6 = var_ +'~sex+age+eTIV+C(diagnosis, Treatment(reference="NC"))'; m6 = smf.glm(formula=f6, data=dat_); r6 = m6.fit();
    f7 = var_ +'~age+sex*eTIV+C(diagnosis, Treatment(reference="NC"))'; m7 = smf.glm(formula=f7, data=dat_); r7 = m7.fit();
    res_list = [r1, r2, r3, r4, r5, r6, r7];
    return res_list
tar_list = ['Cerebellum_Cortex','Cerebellum_White_Matter','Left_Cerebellum_Cortex','Left_Cerebellum_White_Matter',
            'Right_Cerebellum_Cortex','Right_Cerebellum_White_Matter'];

res_list = comp_models(curr_data, tar_list[0])
for item_ in res_list:
    print(item_.summary2())